In [1]:
import sqlite3
import pandas as pd
import json

pd.options.mode.chained_assignment = None
cnx = sqlite3.connect('./game.0802.db')




In [2]:
df = pd.read_sql_query("SELECT * FROM games", cnx)

In [3]:
game_df = pd.read_sql_query("SELECT * FROM game_results", cnx)

In [32]:
mid1

,id,players_name,name,rn
0,ga236647f0eb3,钝钝,UNMI Contractor,1
1,ga236647f0eb3,钝钝,Metal-Rich Asteroid,2
2,ga236647f0eb3,钝钝,Soletta,3
3,ga236647f0eb3,钝钝,Olympus Conference,4
4,ga236647f0eb3,钝钝,Optimal Aerobraking,5
...,...,...,...,...
36111,g711fea3c4229,木木,Algae,47
36112,g711fea3c4229,木木,Windmills,48
36113,g711fea3c4229,木木,Artificial Lake,49
36114,g711fea3c4229,木木,Trans-Neptune Probe,50


In [15]:
# 处理game_df

game_df["scores"] = game_df["scores"].astype(object)
game_df["game_options"] = game_df["game_options"].astype(object)
game_df["createtime"] = pd.to_datetime(game_df["createtime"], dayfirst=True)
store = []
for i in game_df["scores"]:
    tmp = json.loads(i)
    for i in range(len(tmp)):
        tmp[i]['order'] = i+1
    tmp = sorted(tmp,key = lambda e:e.__getitem__('playerScore'),reverse = True) # can optimize to obtain knowledge of order
    for i in range(len(tmp)):
        tmp[i]['position'] = i+1
    store.append(tmp)
df1 = pd.DataFrame(store)
df1.columns = ["player1","player2","player3","player4","player5","player6"]

options = []
for i in game_df["game_options"]:
    if not pd.isna(i):
        options.append(json.loads(i))
    else: options.append({})
df2 = pd.DataFrame(options)

df3 = pd.concat([game_df, df1, df2],axis=1)
df4= df3.drop(columns=['game_options','scores','draftVariant','corporateEra','solarPhaseOption','undoOption','fastModeOption','includeVenusMA','soloTR','removeNegativeGlobalEventsOption','politicalAgendasExtension','requiresMoonTrackCompletion','showTimers'])



In [5]:
# 选择所有已完成数据
df1a = df[df['status']=='finished']

#  关联快照表，取所需字段，即game_df可以通过streamlit选项动态调整
df1b = game_df.merge(df1a,how='left',on=['game_id', 'createtime'], suffixes=['','_drop'], indicator=True).query('_merge == "both"')

df2 = df1b[df1b['players'] == 4]
# 修改需要展开字段的类型为list
false = False
true = True
null = None
df2['game'] = df2['game'].apply(lambda x:eval(x))
mid = pd.json_normalize(df2['game'],record_path=['players', 'playedCards'], meta=['id',  ['players','terraformRating'], ['players','name']], sep='_')




In [17]:
mid1 = mid.loc[:,['id','players_name','name']]
mid1['rn'] = mid1 \
            .groupby(['id','players_name']) \
            .cumcount() + 1
# mid
# TODO  需要关联处理后的game_result表，获取position, gens, createtime等数据, 只能通过merge实现

In [51]:
def getPlayerNumResult(df, player_num = 4):
    """
    主键: game_id, player
    """
    df = df.loc[df['players'] == player_num].reset_index(drop=True)
    for i in range(1, player_num+1):
        player_idx = 'player'+str(i)
        # player_df_pre = df[player_idx].apply(lambda x:eval(x))
        # print(player_idx)
        # player_df = pd.json_normalize(player_df_pre).reset_index(drop=True)
        player_df = pd.json_normalize(df[player_idx]).reset_index(drop=True)
        if i == 1:
            res = pd.concat([df,player_df.reindex(df.index)],axis=1)
            print((res.loc[pd.isna(res['player']) == False]).shape[0])
        else:
            mid = pd.concat([df,player_df.reindex(df.index)],axis=1)
            res = pd.concat([res, mid],axis=0, ignore_index=True)
            # print((mid.loc[pd.isna(mid['player']) == False]).shape[0])
        # df = pd.concat([df, pd.json_normalize(df[player_idx])],axis=1)
    res.drop(['player'+str(i) for i in range(1, 7)], axis=1, inplace=True)
    res['count'] = 1
    return res

def getCardRank(df):
    """
    对card做聚合, 取打出胜率以及打出次数
    """
    res = df.query('rn not in (1,2)') \
    .groupby('name') \
    .agg(
        position = ('position', 'mean'),
        playerScore = ('playerScore', 'mean'),
        generations = ('generations', 'mean'),
        total = ('count', 'sum')
    ) \
    .dropna().sort_values('position').reset_index()
    return res

def getPlayersCardRank(df):
    """
    对card做聚合, 取打出胜率以及打出次数
    """
    res = df.query('rn not in (1,2)') \
    .groupby(['player', 'name']) \
    .agg(
        position = ('position', 'mean'),
        playerScore = ('playerScore', 'mean'),
        generations = ('generations', 'mean'),
        total = ('count', 'sum'),
        sum_position = ('position', 'sum'),
        sum_playerScore = ('playerScore', 'sum'),
        sum_generations = ('generations', 'sum')
    ) \
    .dropna().sort_values('position').reset_index()
    return res

In [33]:
player_df = getPlayerNumResult(df4) # 主键: game_id, player

card_df = mid1.merge(player_df, how='left', left_on=['id', 'players_name'], right_on=['game_id', 'player'], suffixes=['','_drop'], indicator=True).query('_merge == "both"')


2587


In [42]:
def getPlayerCardRank(df, player):
    ff1 = df.query('player == "%s"'%(player))
    return getCardRank(ff1)

In [49]:
card_df_group = getCardRank(card_df)
getPlayersCardRank(card_df).to_csv('playersCardRank.csv', index=False)

In [45]:
getPlayerCardRank(card_df, '钝钝')

,name,position,playerScore,generations,total
0,Import of Advanced GHG,1.0,92.000000,7.000000,1
1,Self-replicating Robots,1.0,98.666667,7.333333,3
2,Fueled Generators,1.0,78.000000,7.000000,1
3,Solar Power,1.0,99.000000,7.000000,1
4,Self-Sufficient Settlement,1.0,85.000000,6.000000,1
...,...,...,...,...,...
358,Sponsored Mohole,3.5,79.000000,7.000000,2
359,Regolith Eaters,4.0,68.000000,7.000000,1
360,Heather,4.0,68.000000,7.000000,1
361,Herbivores,4.0,66.000000,6.000000,1


In [97]:
false = False
true = True
player_df_new = df2['game'].apply(lambda x:eval(x))

In [143]:
player_df_new

6950    {'id': 'gb53baf3212e3', 'players': [{'name': '...
7020    {'id': 'g32498849988e', 'players': [{'name': '...
7068    {'id': 'gd4c2a4f2bbf3', 'players': [{'name': '...
7083    {'id': 'gb53baf3212e3', 'players': [{'name': '...
7089    {'id': 'g3855ccb42bec', 'players': [{'name': '...
Name: game, dtype: object

In [156]:
# method1
mid = pd.json_normalize(player_df_new,record_path=['players', 'playedCards'], meta=[['players','terraformRating'], ['players','name']], sep='_')
mid
# pd.json_normalize(mid.iloc[0],record_path=['playedCards'])


,name,resourceCount,bonusResource,players_terraformRating,players_name
0,Biolab,NaN,NaN,33,米饭
1,Aerospace Mission,NaN,NaN,33,米饭
2,Nitrate Reducers,NaN,NaN,33,米饭
3,Psychrophiles,1.0,NaN,33,米饭
4,Invention Contest,NaN,NaN,33,米饭
...,...,...,...,...,...
115,Underground City,NaN,NaN,56,You
116,Algae,NaN,NaN,56,You
117,Sub-Crust Measurements,NaN,NaN,56,You
118,Galilean Waystation,NaN,NaN,56,You


In [120]:
# pd.json_normalize(player_df_new,record_path=['players'],meta=['id'])

# mid = pd.json_normalize(player_df_new,record_path=['players'])
mid = pd.json_normalize(player_df_new)['players']
# pd.json_normalize(mid['players'],record_path=['playedCards'],meta=['name',['_game','id']], record_prefix='*')
# pd.json_normalize(mid['players'],record_path=['playedCards'])

In [133]:
mid

0    [{'name': 'You', 'color': 'red', 'beginner': F...
1    [{'name': '米饭', 'color': 'red', 'beginner': Fa...
2    [{'name': 'You', 'color': 'red', 'beginner': F...
3    [{'name': 'You', 'color': 'red', 'beginner': F...
4    [{'name': 'You', 'color': 'red', 'beginner': F...
Name: players, dtype: object

In [134]:
mid = mid.tail(1)

In [139]:
mid.iloc[0]

[{'name': 'You',
  'color': 'red',
  'beginner': False,
  'handicap': 0,
  '_game': {'id': 'g3855ccb42bec'},
  'corpCard': {'name': 'Saturn Systems'},
  'pickedCorporationCard': {'name': 'Saturn Systems'},
  'terraformRating': 56,
  'hasIncreasedTerraformRatingThisGeneration': True,
  'terraformRatingAtGenerationStart': 14,
  'megaCredits': 63,
  'megaCreditProduction': 7,
  'steel': 4,
  'steelProduction': 4,
  'titanium': 1,
  'titaniumProduction': 1,
  'plants': 3,
  'plantProduction': 11,
  'energy': 3,
  'energyProduction': 3,
  'heat': 15,
  'heatProduction': 13,
  'titaniumValue': 3,
  'steelValue': 2,
  'canUseHeatAsMegaCredits': False,
  'actionsTakenThisRound': 0,
  'actionsThisGeneration': [],
  'undoing': False,
  'exited': False,
  'canExit': False,
  'corpInitialActionDone': False,
  'corp2InitialActionDone': False,
  'dealtCorporationCards': [],
  'dealtProjectCards': [],
  'dealtPreludeCards': [],
  'cardsInHand': [{'resourceCount': 0, 'name': 'Martian Zoo'},
   {'resou

In [140]:
pd.json_normalize(mid.iloc[0])

,name,color,beginner,handicap,terraformRating,hasIncreasedTerraformRatingThisGeneration,terraformRatingAtGenerationStart,megaCredits,megaCreditProduction,steel,...,heatProductionStepsIncreasedThisGeneration,id,_game.id,corpCard.name,pickedCorporationCard.name,timer.sumElapsed,timer.startedAt,timer.running,timer.afterFirstAction,timer.lastStoppedAt
0,You,red,False,0,56,True,14,63,7,4,...,0,p78988c7af87e,g3855ccb42bec,Saturn Systems,Saturn Systems,1430484,1627868526517,False,True,1627868527001


In [141]:
pd.json_normalize(mid.iloc[0],record_path=['playedCards'])

,name,resourceCount,bonusResource
0,Mohole Excavation,NaN,NaN
1,Business Empire,NaN,NaN
2,Regolith Eaters,0.0,NaN
3,Convoy From Europa,NaN,NaN
4,Subterranean Reservoir,NaN,NaN
5,Lichen,NaN,NaN
6,Mohole Area,NaN,NaN
7,Trade Envoys,NaN,NaN
8,Deep Well Heating,NaN,NaN
9,Peroxide Power,NaN,NaN


In [64]:
false = False
true = True
player_df_pre = df2['game'].apply(lambda x:eval(x))
# print(player_idx)
player_df = pd.json_normalize(player_df_pre).reset_index(drop=True)
player_df = player_df.loc[:,['id', 'players']]
# pd.DataFrame([flatten_json(x) for x in eval(df.iloc[0,2])])

In [67]:
# player_df['players'] = player_df['players'].apply(lambda x:eval(x))
player_df2 = player_df.explode('players')
# player_df3a = player_df2['players'].apply(lambda x:eval(x))
# print(player_idx)
player_df3 = pd.json_normalize(player_df2).reset_index(drop=True)

In [91]:
# player_df2['players'] = eval(player_df2['players'])
player_df2['players']


0    {'name': 'You', 'color': 'red', 'beginner': Fa...
1    {'name': '米饭', 'color': 'red', 'beginner': Fal...
1    {'name': '小麦', 'color': 'green', 'beginner': F...
2    {'name': 'You', 'color': 'red', 'beginner': Fa...
3    {'name': 'You', 'color': 'red', 'beginner': Fa...
4    {'name': 'You', 'color': 'red', 'beginner': Fa...
Name: players, dtype: object

In [89]:
player_df2['players'] = player_df2['players'].astype(str).apply(lambda x:eval(x))

In [93]:
player_df3 = pd.json_normalize(player_df2['players'])[:,'']

,name,color,beginner,handicap,terraformRating,hasIncreasedTerraformRatingThisGeneration,terraformRatingAtGenerationStart,megaCredits,megaCreditProduction,steel,...,waitingFor.options,userId,corpCard.name,pickedCorporationCard.name,corpCard.tags,corpCard.startingMegaCredits,corpCard.cardType,pickedCorporationCard.tags,pickedCorporationCard.startingMegaCredits,pickedCorporationCard.cardType
0,You,red,False,0,14,False,14,0,0,0,...,"[{'title': 'Select corporation', 'buttonLabel'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,米饭,red,False,0,33,True,20,58,21,1,...,NaN,d484c04d1ffb,Valley Trust (breakthrough),Valley Trust (breakthrough),NaN,NaN,NaN,NaN,NaN,NaN
2,小麦,green,False,0,52,True,20,71,15,0,...,NaN,dfc7613bd132,Interplanetary Cinematics (breakthrough),Interplanetary Cinematics (breakthrough),[building],50.0,corporation,[building],50.0,corporation
3,You,red,False,0,62,True,14,82,7,12,...,NaN,NaN,EcoLine,EcoLine,NaN,NaN,NaN,NaN,NaN,NaN
4,You,red,False,0,37,True,14,56,11,12,...,NaN,NaN,CrediCor,CrediCor,NaN,NaN,NaN,NaN,NaN,NaN
5,You,red,False,0,56,True,14,63,7,4,...,NaN,NaN,Saturn Systems,Saturn Systems,NaN,NaN,NaN,NaN,NaN,NaN
